## C S 329E HW 6

# KNN 

# HW 6 Pair 1

## Pamela Nguyen and Daniel Carlson


For this week's homework we are going explore one new classification technique:

  - k nearest neighbors

We are using a different version of the Melbourne housing data set, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

At the end of this homework, I expect you to understand how to build and use a kNN model, and practice your data cleaning and data preparation skills. 

In [ ]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
%matplotlib inline

# Starting off loading a training set
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')

## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formatting that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. Please use the python [reference for time](https://docs.python.org/3/library/time.html) to help you do the string to Unix time conversion. 

In [ ]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
  date = np.array(d.split('/'))
  yr, mon, day = int(date[2]),int(date[1]),int(date[0])
  # Manually adjust the data to 2000's format
  if yr == 16:
    yr=2016
  if yr == 17:
    yr=2017
  date = yr, mon, day,0,0,0
  date = calendar.timegm(date) #convert to Unix timestamp from GMT
  #print(date)
  #print(np.type(date))
  return date


In [ ]:
df_melb['unixtime'] = df_melb['Date'].apply(lambda x : standardize_date(x)) 

In [ ]:
df_melb = df_melb.drop(columns="Date")
print("The min unixtime is {:d} and the max unixtime is {:d}".format(df_melb['unixtime'].min(),df_melb['unixtime'].max()))

The min unixtime is 1454544000 and the max unixtime is 1506124800


## Q2 Use Imputation to fill in missing values
kNN doesn't work when the attributes are not valid for all of the attribute columns, so fill in all the missing values in `df_melb` with the mean of that column.  Save the mean of each column in a dictionary, `dict_imputation`, whose key is the attribute column name, so we can apply the same imputation to the test set later. Show your `dict_imputation` dictionary and the head of your `df_melb` dataframe.  The target classfication (aka the class label) is stored in the column `'Type'`, so we are going to define a variable target_col so we can reference the target_col using a variable. (hint: during imputation you skip the target column)

In [ ]:
target_col = 'Type'

In [ ]:
dict_imputation = dict()
for col in df_melb.columns:
    if col != target_col:
      col_mean = np.mean(df_melb[col])
      dict_imputation.update({col:col_mean})
      df_melb[col].fillna(value=col_mean,inplace=True)#fill na values of each respective col with its mean

In [ ]:
dict_imputation

{'Bathroom': 1.44,
 'BuildingArea': 121.7832,
 'Car': 1.503006012024048,
 'Distance': 10.524599999999985,
 'Landsize': 638.91,
 'Postcode': 3113.122,
 'Price': 932558.7,
 'Rooms': 2.71,
 'YearBuilt': 1970.9417475728155,
 'unixtime': 1485178502.4}

In [ ]:
df_melb.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,2,h,399000,8.7,3032,1,1.0,904,53.0000,1985.000000,1462579200
1,3,h,1241000,13.9,3165,1,1.0,643,121.7832,1970.941748,1472342400
2,2,u,550000,3.0,3067,1,1.0,1521,121.7832,1970.941748,1499472000
3,3,u,691000,8.4,3072,1,1.0,170,121.7832,1970.941748,1498262400
4,2,u,657500,4.6,3122,1,1.0,728,73.0000,1965.000000,1479513600


## Q3 Normalize all the attributes to be between [0,1]
Normalize all the attribute columns in `df_melb` so they have a value between zero and one (inclusive). Save the (min,max) tuple used to normalize to a dictionary, `dict_normalize`, so we can apply it to the test set later.  The dataframe `df_melb` is now your "model" that you can use to classify new data points. (hint: during normalization you skip the class label column)

In [ ]:
dict_normalize = dict()

for col in df_melb.columns:
  if col != target_col:
    col_min,col_max = np.min(df_melb[col]), np.max(df_melb[col])
    col_norm = col_min, col_max
    dict_normalize.update({col:col_norm})
    df_melb[col] = (df_melb[col] - col_min)/(col_max-col_min) #normalize cols
    

In [ ]:
dict_normalize

{'Bathroom': (0, 4),
 'BuildingArea': (0.0, 475.0),
 'Car': (0.0, 4.0),
 'Distance': (0.7, 47.3),
 'Landsize': (0, 41400),
 'Postcode': (3002, 3810),
 'Price': (291000, 5020000),
 'Rooms': (1, 6),
 'YearBuilt': (1890.0, 2015.0),
 'unixtime': (1454544000, 1506124800)}

In [ ]:
df_melb.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,0.2,h,0.022838,0.171674,0.037129,0.25,0.25,0.021836,0.111579,0.760000,0.155779
1,0.4,h,0.200888,0.283262,0.201733,0.25,0.25,0.015531,0.256386,0.647534,0.345059
2,0.2,u,0.054768,0.049356,0.080446,0.25,0.25,0.036739,0.256386,0.647534,0.871022
3,0.4,u,0.084584,0.165236,0.086634,0.25,0.25,0.004106,0.256386,0.647534,0.847571
4,0.2,u,0.077501,0.083691,0.148515,0.25,0.25,0.017585,0.153684,0.600000,0.484087


## Q4 Load in the Test data and prep it for classification
Everything we did to our "train" set, we need to now do in our "test" set. 

In [ ]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')

In [ ]:
df_test['unixtime'] = df_test['Date'].apply(lambda x : standardize_date(x)) 
df_test = df_test.drop(columns="Date")
print("The min unixtime is {:d} and the max unixtime is {:d}".format(df_test['unixtime'].min(),df_test['unixtime'].max()))

The min unixtime is 1454544000 and the max unixtime is 1506124800


In [ ]:
# Your code here for imputation - must use dictionary from above!
for col in df_test.columns:
  if col != target_col:
    df_test[col].fillna(value=dict_imputation[col],inplace=True)
df_test.head()        

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,3,h,1116000,17.9,3192,1,2.0,610,121.7832,1970.941748,1498867200
1,3,h,2030000,11.2,3186,2,2.0,366,121.7832,1970.941748,1472342400
2,3,h,1480000,10.7,3187,2,2.0,697,143.0000,1925.000000,1478476800
3,3,u,1203500,12.3,3166,2,2.0,311,127.0000,2000.000000,1495843200
4,3,h,540000,14.7,3030,2,2.0,353,135.0000,2011.000000,1504396800


In [ ]:
# Your code here for scaling - must use dictionary from above!
for col in df_test.columns:
  if col != target_col:
    df_test[col] = (df_test[col]-dict_normalize[col][0]) / (dict_normalize[col][1]-dict_normalize[col][0])
df_test.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,0.4,h,0.174455,0.369099,0.235149,0.25,0.5,0.014734,0.256386,0.647534,0.859296
1,0.4,h,0.367731,0.225322,0.227723,0.50,0.5,0.008841,0.256386,0.647534,0.345059
2,0.4,h,0.251427,0.214592,0.228960,0.50,0.5,0.016836,0.301053,0.280000,0.463987
3,0.4,u,0.192958,0.248927,0.202970,0.50,0.5,0.007512,0.267368,0.880000,0.800670
4,0.4,h,0.052654,0.300429,0.034653,0.50,0.5,0.008527,0.284211,0.968000,0.966499


## Q5 Write the kNN classifier function
Your function `knn_class`, should take five parameters, the training dataframe (that includes the target column), the hyper parameter `k`, the name of the target column, a single observation row (a series or attributes the same length as the attributes in `df_train`) of the test dataframe, and a boolean `use_weighted_vote`.  When `use_weighted_vote` is set to true, use weighted voting, otherwise use majority voting. We are assuming that the parameter `df_train` contains all of the attributes, and the target class in the same dataframe. The function returns the predicted target classification for that observation. To find the distance between the single observation and the training data frame you should use the [L2 norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)

In [ ]:
def weighted_vote(d):
  return 1/d**2

def majority_vote():
  return

def get_distance(df_train, observation):
  diff=[]
  for index, row in df_train.iterrows():
    diff.append(row-observation)
  dist = np.sqrt(np.square(diff).sum(axis=1).sum(axis=1))
  return dist

# def predictions(df_train):
#   num_pred, num_features = df_train.shape
#   preds = np.array([])
#   for i in range(num_pred):
#     preds = np.append(preds, get_pred)

# Return the predicted target classification for the observation
def knn_class(df_train, k, target_col, observation, use_weighted_vote):
  df_train_copy = df_train.copy()
  df_train_copy[~df_train_copy.isin(observation)].dropna()
  dist = get_distance(df_train_copy[df_train_copy.columns.drop(target_col)], observation[observation.columns.drop(target_col)])
  df_train_copy['Norm'] = dist
  df_dist = df_train_copy.sort_values(by=['Norm'], axis=0, ascending=True)
  df_dist = df_dist.reset_index(drop=True)
  df_dist = df_dist.loc[:(k-1)]
  print(df_dist)
  dict_dist = {}
  if use_weighted_vote == False:
    for index, row in df_dist.iterrows():
      if row['Type'] in dict_dist:
        dict_dist[row['Type']] += 1
      else:
        dict_dist[row['Type']] = 1
    return(max(dict_dist, key=dict_dist.get))
  else:
    for index, row in df_dist.iterrows():
      if row[target_col] in dict_dist:
        dict_dist[row[target_col]] += 1/((row['Norm'])**2)
      else:
        dict_dist[row[target_col]] = 1/((row['Norm'])**2)
    return(max(dict_dist, key=dict_dist.get))

In [ ]:
# Soln from class
# Return the predicted target classification for the observation
def knn_class(df_train, k, target_col, observation, use_weighted_vote):
  df_train_copy = df_train.copy()
  df_train_attributes = df_train.drop(columns = [target_col])
  df_train['distance'] = (df_train_attributes - observation).apply(np.linalg.norm, axis=1)
  
  df_train = df_train.sort_values(by='distance').reset_index(drop=True)
  df_k = df_train[0:k].copy()
  df_k['weight'] = 1.0/(df_k['distance']**2)

  if use_weighted_vote:
    votes = df_k.groupby(target_col)['weight'].sum()
    prediction = votes.index[np.argmax(votes)]
  else:
    prediction = df_k[target_col].value_counts().index[0]
  return prediction


## Q6 Compute the accuracy using different k values
For each value of $k$ in the set $\{1,3,13,25,50,100\}$ calculate the class prediction for each oberservation in the test set, and the overall accuracy of the classifier.  Plot the accuracy as a function of $k$ when `use_weighted_vote` is `True` and when `use_weighted_vote` is `False`.

Which value of $k$ would you chose, and would you use weighted voting or majority voting?

Note, this took 5 minutes on my computer. 

In [ ]:
poss_k = [1,3,13,25,50,100] # possible k's
pred = {}
pred_weighted = {}
for k in poss_k:
  obs_list = []
  obs_list_weighted = []
  for j in range(len(df_test)):
    obs_list.append(knn_class(df_melb, k, target_col, df_test.iloc[[j]], False)) #appending is more computationally expensive then creating list of zeros and reassigning
    obs_list_weighted.append(knn_class(df_melb, k, target_col, df_test.iloc[[j]], True))
  pred[k] = obs_list
  pred_weighted[k] = obs_list_weighted

print(pred, pred_weighted)

acc_k_majority = list(np.zeros(len(poss_k))) # Accuracy for each value of k using majority voting
acc_k_weighted = list(np.zeros(len(poss_k))) # Accuracy for each value of k using weighted voting

# df_dist = pd.DataFrame()
# for k in poss_k:
#   acc_k_majority = np.append(acc_k_majority, knn_class(df_melb, k, target_col, df_test, True))
#   acc_k_weighted = np.append(acc_k_weighted, knn_class(df_melb, k, target_col, df_test, False))

{1: ['h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h'], 3: ['h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 

In [ ]:
df_test_list = df_test['Type'].tolist()
correct = []
correct_weighted = []
for k in pred:
  correct.append(sum(x == y for x, y in zip(pred[k], df_test_list)))
  correct_weighted.append(sum(x == y for x, y in zip(pred_weighted[k], df_test_list)))

acc = []
for i in correct:
  acc.append(i/100)

acc_weighted = []
for j in correct_weighted:
  acc_weighted.append(j/100)

print(acc)
print(acc_weighted)

[0.67, 0.71, 0.7, 0.71, 0.67, 0.69]
[0.67, 0.73, 0.71, 0.7, 0.7, 0.7]


Text(0, 0.5, 'Accuracy')

In [ ]:
# plot code here
plt.plot(poss_k, acc, label='Unweighted')
plt.plot(poss_k, acc_weighted, label='Weighted')
plt.legend(loc="upper right")
plt.title('Accuracy of kNN')
plt.xlabel('k')
plt.ylabel('Accuracy')
# Accuracy is clearly better when weighting the neighbors. Accuracy is highest when k=1.
# For best results with this data

➡️ Answer containing your analysis of the I would choose $k = <value> $ and voting scheme because _reasons_ here ⬅️

For best results with this data, use k=1 weighted kNN.
Accuracy is clearly better when weighting the neighbors. Accuracy is highest when k=1 with this voting scheme.